In [1]:
import torch
import torchvision
import torch.nn as nn
import time
import json
import datetime
import matplotlib.pyplot as plt

In [2]:
%cd
from DeepLearning.Project2.data_loading_preparation import load_audio_dataloaders_validation

/home/kacper


In [3]:
def get_generic_classifier(input_size, hidden_size, output_size):
    classifier = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, output_size),
    )
    return classifier

class RecurrentSimple(nn.Module):
    def __init__(
        self,
        input_size, 
        hidden_size,
        num_layers,
        num_classes = 12,
        avgpool_dim = 32,
        add_dropout=True,
        classifier_size = 512
    ):       
        super().__init__()
        self.normalization = nn.BatchNorm1d(input_size)
        self.features = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.avgpool = nn.AdaptiveAvgPool1d(avgpool_dim)
        self.classifier = get_generic_classifier(
            hidden_size * avgpool_dim,
            classifier_size,
            num_classes
        )

    def forward(self, x):
        x = self.normalization(x)
        x = x.mT
        x, _ = self.features(x)
        x = x.mT
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
class LSTMSimple(nn.Module):
    def __init__(
        self,
        input_size, 
        hidden_size,
        num_layers,
        num_classes = 12,
        avgpool_dim = 32,
        classifier_size = 512,
        add_dropout=True
    ):       
        super().__init__()
        self.normalization = nn.BatchNorm1d(input_size)
        self.features = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.avgpool = nn.AdaptiveAvgPool1d(avgpool_dim)
        self.classifier = get_generic_classifier(
            hidden_size * avgpool_dim,
            classifier_size,
            num_classes
        )

    def forward(self, x):
        x = self.normalization(x)
        x = x.mT
        x, _ = self.features(x)
        x = x.mT
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
def eval_accuracy(model, dataloader, reccurrent, training_device='cuda'):
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.float().to(training_device), labels.float().to(training_device)
            yhat = model(inputs)
            pred = torch.argmax(yhat, dim=1)

            
            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels)).item()
    return correct/all_so_far

In [5]:
def backup_to_ram(model):
    from copy import deepcopy
    return deepcopy(model).cpu()

class EarlyStopper:
    def __init__(self, patience = 3, backup_method=backup_to_ram):
        self.patience = patience
        self.current = 0
        
        self.backup_method = backup_method
        
        self.best_backup = None
        self.best_accuracy = 0.

    def should_continue(self, accuracy, model = None):
        if self.best_accuracy < accuracy:
            self.current = 0
            self.best_accuracy = accuracy
            if model is not None:
                self.best_backup = self.backup_method(model)
            return True
        
        self.current += 1
        
        if self.current >= self.patience:
            return False
        return True

In [6]:
def run_experiment(experiment_name, train_func, run, train_params=None):
    path = f"experiments_rnn/{experiment_name}_run_{run}_"
    print("Running experiment for ", path[:-1])
    
    import os
    try:
        if os.stat(path + "report.json").st_size != 0:
            print("Report exists already for " + path[:-1] + ". Skipping...")
            return
    except OSError:
        pass
    
    model, trajectory, validation_accuracy = train_func(train_params)
    
    with open(path + "report.json", "w") as f:
        json.dump(
            {
                "name": experiment_name,
                "train_params": train_params,
                "run": run,
                "best_accuracy_validation": validation_accuracy,
                "time_generated": datetime.datetime.now().isoformat(),
                "trajectory": trajectory
            },
            f
        )
    torch.save(model, path + "model.pt")

In [7]:
training_device = "cuda"
device = "cuda"
max_epochs = 250

def oneRNN(train_params):  
    train, test, val = load_audio_dataloaders_validation(bs=128)
    criterion_weights = torch.tensor([1.] * 11 + [0.1]).to(device)
    model = RecurrentSimple(**train_params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 10)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy

def oneLSTM(train_params):  
    train, test, val = load_audio_dataloaders_validation(bs=128)
    criterion_weights = torch.tensor([1.] * 11 + [0.1]).to(device)
    model = LSTMSimple(**train_params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 10)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy

def equalRNN(train_params):  
    train, test, val = load_audio_dataloaders_validation(bs=128)
    model = RecurrentSimple(**train_params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 10)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy

def equalLSTM(train_params):  
    train, test, val = load_audio_dataloaders_validation(bs=128)
    model = LSTMSimple(**train_params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 10)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()

        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy

In [8]:
param_grid = {
    "tiny": {
        "hidden_size": 40,
        "input_size": 20,
        "num_layers": 1,
    },
    "tinydouble": {
        "hidden_size": 40,
        "input_size": 20,
        "num_layers": 2,
    },
    "tinytriple": {
        "hidden_size": 40,
        "input_size": 20,
        "num_layers": 3,
    },
    "wide": {
        "hidden_size": 80,
        "input_size": 20,
        "num_layers": 1,
    },
    "widedouble": {
        "hidden_size": 80,
        "input_size": 20,
        "num_layers": 2,
    }
}


experiment_list = [
    (
        f"trainer_{trainer.__name__}_params_{param_name}", 
        trainer,
        str(run),
        param
    )
    for run in range(1, 11) 
    for trainer in [oneRNN, oneLSTM]
    for param_name, param in param_grid.items()
]

In [9]:
for experiment in experiment_list:
    print(
        "Time:", datetime.datetime.now().isoformat(),
        "Experiment:", experiment[0]
    )
    try:
        run_experiment(*experiment)
    except Exception as e:
        print("Error occured, skipping...\n", repr(e))

Time: 2023-04-16T10:48:11.733273 Experiment: trainer_oneRNN_params_tiny
Running experiment for  experiments_rnn/trainer_oneRNN_params_tiny_run_1
Report exists already for experiments_rnn/trainer_oneRNN_params_tiny_run_1. Skipping...
Time: 2023-04-16T10:48:11.734560 Experiment: trainer_oneRNN_params_tinydouble
Running experiment for  experiments_rnn/trainer_oneRNN_params_tinydouble_run_1
Report exists already for experiments_rnn/trainer_oneRNN_params_tinydouble_run_1. Skipping...
Time: 2023-04-16T10:48:11.734592 Experiment: trainer_oneRNN_params_tinytriple
Running experiment for  experiments_rnn/trainer_oneRNN_params_tinytriple_run_1
Report exists already for experiments_rnn/trainer_oneRNN_params_tinytriple_run_1. Skipping...
Time: 2023-04-16T10:48:11.734626 Experiment: trainer_oneRNN_params_wide
Running experiment for  experiments_rnn/trainer_oneRNN_params_wide_run_1
Report exists already for experiments_rnn/trainer_oneRNN_params_wide_run_1. Skipping...
Time: 2023-04-16T10:48:11.734650

Epoch: 1, Accuracy on validation set: 0.19982902329557597
Epoch: 2, Accuracy on validation set: 0.23338320153879033
Epoch: 3, Accuracy on validation set: 0.2949348151314383
Epoch: 4, Accuracy on validation set: 0.3714468903611883
Epoch: 5, Accuracy on validation set: 0.3584099166488566
Epoch: 6, Accuracy on validation set: 0.4238085060910451
Epoch: 7, Accuracy on validation set: 0.4045736268433426
Epoch: 8, Accuracy on validation set: 0.46056849754220985
Epoch: 9, Accuracy on validation set: 0.4556529172900192
Epoch: 10, Accuracy on validation set: 0.43876896772814705
Epoch: 11, Accuracy on validation set: 0.4761701218209019
Epoch: 12, Accuracy on validation set: 0.5069459286172259
Epoch: 13, Accuracy on validation set: 0.4355631545201966
Epoch: 14, Accuracy on validation set: 0.5024577901260954
Epoch: 15, Accuracy on validation set: 0.4900619790553537
Epoch: 16, Accuracy on validation set: 0.5099380209446462
Epoch: 17, Accuracy on validation set: 0.4939089549048942
Epoch: 18, Accuracy

Epoch: 18, Accuracy on validation set: 0.4838026677958924
Epoch: 19, Accuracy on validation set: 0.4088503070082575
Epoch: 20, Accuracy on validation set: 0.482108829134025
Epoch: 21, Accuracy on validation set: 0.4946008892652975
Epoch: 22, Accuracy on validation set: 0.4361634554308702
Epoch: 23, Accuracy on validation set: 0.44060978191827227
Epoch: 24, Accuracy on validation set: 0.4920601312724963
Epoch: 25, Accuracy on validation set: 0.45754816853694685
Epoch: 26, Accuracy on validation set: 0.4647469828498835
Epoch: 27, Accuracy on validation set: 0.42134236713952994
Epoch: 28, Accuracy on validation set: 0.48126190980309125
Epoch: 29, Accuracy on validation set: 0.40546262968452257
Epoch: 30, Accuracy on validation set: 0.45543087020961254
Epoch: 31, Accuracy on validation set: 0.4435739995765403
Early stop
Time: 2023-04-16T11:06:07.076786 Experiment: trainer_oneLSTM_params_widedouble
Running experiment for  experiments_rnn/trainer_oneLSTM_params_widedouble_run_10
Epoch: 1, Ac